In [1]:
from datasets import load_dataset
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer

In [2]:
dataset = load_dataset("nsmc")

Using custom data configuration default
Reusing dataset nsmc (/home/cleaning/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
num_classes = 2
train_dataset = dataset["train"].shuffle(seed=42).select(range(42 * num_classes))
eval_dataset = dataset["test"]

Loading cached shuffled indices for dataset at /home/cleaning/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-dcd892fe1da31a64.arrow


In [4]:
train_dataset['document']

['For Carl.칼 세이건으로 시작해서 칼 세이건으로 끝난다.',
 '모든 면에서 너무 좋고, 특히 동양적이고',
 '달콤한 꿀과 톡쏘는 칠리의 만남',
 '토비 맥과이어가 그립다.',
 '전두엽절제술 ㅠㅠ 넘싫다',
 '재밌게보고있지만 점점식상해지는부분도없지않음',
 '전작의 명성에 똥칠하는 망작',
 '산으로 가는 스토리',
 '이것도영화라고ㅋㅋ',
 '프랑스 교도소 간수 죽여버리고 싶었다',
 '미이케 다케시 영화는 복불복임 ㅋㅋㅋ 개쩌는영화는 진짜 개쩌는데... 이건 영~~',
 '한쪽을 잃은 사람들은 현재와과거에서 상통하는걸 모르는 설탕가게(CJ)영화간부들',
 '유카와 센세~!!최고로 매력적인 캐릭터..후쿠야마 마사하루 말고 그누가 어울릴까..',
 '유사 소재 한국 영화 중 비교적 지루하고 임팩트 없음. 그나마 김윤석의 조연시절 보는 재미와 유지태의 깔끔한 연기를 보는 재미 정도 있음.',
 '진짜 재미있을 것 같은데, 부디 우라나라 애니메이션에 커다란 발전이 있기를.',
 '일본하고 독일을 엮지마라 적어도 일본은 나치스처럼 인종학살은 하지않았다.',
 '매주 수목을 기다리며 본방사수한 드라마 ..작가 배우 모두 훌륭합니다',
 '질질 끌면서 짜증맛 더 해가는 쓰레기 드라마작가야 언제 반전 보여 줄래이제 얼마 안 남았잖아',
 '잔잔하고 재미도 있어서 좋았다',
 '인생은 결국 섹스문제다',
 '클라이브 오웬, 모니카 벨루치 나오는 영화는 좀 지루함. 이것도 즐길만한 수준의 영화가 아님. 유치함.',
 '새로운 시도도 없으면서, 일관성도 없는, 잡스러운 영화!',
 '치킨집 사장들이 봐야하는영화 근데 넘 맛있는걸 우짜지..ㅠㅠ',
 '동생과의관계를 느끼게해주었음 교육적이며 재미까지갖춘 꽤괜찮은애니였음 살짝눈물도 .. .ㅎ',
 '팬티웃김ㅋㅋㅋㅋ 그런데 그런데....... 못생김 ㄱㅡ',
 '배우들의 연기는 나쁘지 않지만 급조된 저예산 무비의 한계랄까? 식상한 내용인데 군데군데 개연성 무너지는 씬들이 많다',
 '짝퉁의 말로 견자단 비스무리한 넘

In [5]:
model = SetFitModel.from_pretrained("ddobokki/klue-roberta-base-nli-sts")

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [6]:
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss_class=CosineSimilarityLoss,
    metric="accuracy",
    batch_size=16,
    num_iterations=20, # The number of text pairs to generate for contrastive learning
    num_epochs=1, # The number of epochs to use for constrastive learning
    column_mapping={'document': 'text', "label": "label"} # Map dataset columns to text/label expected by trainer
)


In [7]:
trainer.train()
metrics = trainer.evaluate()

Applying column mapping to training dataset
***** Running training *****
  Num examples = 3360
  Num epochs = 1
  Total optimization steps = 210
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/210 [00:00<?, ?it/s]

Applying column mapping to evaluation dataset
***** Running evaluation *****


In [8]:
import random


In [10]:
metrics

{'accuracy': 0.80988}

In [11]:
from sklearn.metrics.pairwise import cosine_distances

In [12]:
def get_sim(vec1,vec2):
    return 1-cosine_distances(vec1,vec2)

In [19]:
idx1 = random.randint(0,len(eval_dataset))
idx2 = random.randint(0,len(eval_dataset))
text1 =eval_dataset[idx1]['document']
text2 =eval_dataset[idx2]['document']
label1 =eval_dataset[idx1]['label']
label2 =eval_dataset[idx2]['label']
print(f"text1 :{text1} - label:{label1}")
print(f"text2 :{text2} - label:{label2}")
print(f'is same: {label1 == label2}')

vec1 = model.model_body.encode([text1])
vec2 = model.model_body.encode([text2])

print(f'similarity: {get_sim(vec1,vec2)}')

text1 :첨으로 평점이라는걸 남겨본다 . 이런 쓰렉이는 처음 본다 진짜 욕나오네 - label:0
text2 :라오스 3번 다녀왔어요....어제 9회보니 방비엥의 낯익은정겨운 곳들이 다시 눈에 들어오니 마음이 울컥허더군요 ㅎㅎ꽃보다 시리즈 전편 열심히 애청하고 있습니다제작진과 출연자 여러분들 항상 건강하시고 좋은 방송 부탁드려요 - label:1
is same: False
similarity: [[0.21515155]]
